In [1]:
import os
from concurrent.futures import ProcessPoolExecutor
# set up Whitebox Environment, get available functions
from whitebox_workflows import WbEnvironment
from scripts.constants import create_scenes_df, dem_fill
from scripts.constants import flow_direction, flow_accumulation
import shutil
import multiprocessing
import rasterio
from rasterio.plot import show

In [2]:
logical_cores = multiprocessing.cpu_count()
# determine number of threads to use for multiprocessing
num_workers = int(logical_cores * 0.75)  # rounds down in case not a whole number
print(f'Number of threads to use: {num_workers}')

wbe = WbEnvironment()

scenes_df = create_scenes_df()

Number of threads to use: 7


In [3]:
data_folder = 'data'
input_dem = os.path.join(data_folder, scenes_df.at[0, 'scene_folder'],scenes_df.at[0, 'input_dem'])
print(input_dem)

temp_dir = 'data/temp_dir'

if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)
else:
    print(f'{temp_dir} already exists')

data/S1A_IW_20250205T233956_DVP_RTC10_G_gdufem_246A/S1A_IW_20250205T233956_DVP_RTC10_G_gdufem_246A_dem.tif
data/temp_dir already exists


In [4]:
wbe.verbose = True
wbe.work_directory = temp_dir
print(wbe.work_directory)

# # copy the dem to the directory
dem_copy = f'{temp_dir}/dem_copy.tif'
print(input_dem)
shutil.copyfile(input_dem, dem_copy)

data/temp_dir
data/S1A_IW_20250205T233956_DVP_RTC10_G_gdufem_246A/S1A_IW_20250205T233956_DVP_RTC10_G_gdufem_246A_dem.tif


'data/temp_dir/dem_copy.tif'

TODO, fill missing data.


Call `dem_fill` function from *constants.py* script.  This will
 - find pits
 - fill depressions
 - find flats on flow
 - fix flats

Process can take 10 minutes or so.

In [5]:
# I'm setting this up to run on multiple CPUs, even though here we are only using one.  
# The idea is that in the future, I may do this on several files
if not os.path.exists('data/temp_dir/filled_dem.tif'): 
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        future = executor.submit(dem_fill, dem_copy, temp_dir)
        input_dem_raster = future.result()
else:
    print('Filled Dem Tiff file already exists')  

Filled Dem Tiff file already exists


#### Generate Flow direction
Create a flow direction model from the Filled Dem we just created.
Runs in seconds

In [6]:
filled_dem = f'{temp_dir}/filled_dem.tif'

if not os.path.exists('data/temp_dir/dem_flow.tif'):
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        future = executor.submit(flow_direction, filled_dem, temp_dir)
else:
    print(f'flow_direction already exists.')

Working directory: data/temp_dir
Start d8_pointer function . . . 
********************************************
* Welcome to d8_pointer                    *
* Powered by Whitebox Workflows for Python *
* www.whiteboxgeo.com                      *
********************************************
Progress: 0%
Progress: 1%
Progress: 2%
Progress: 3%
Progress: 4%
Progress: 5%
Progress: 6%
Progress: 7%
Progress: 8%
Progress: 9%
Progress: 10%
Progress: 11%
Progress: 12%
Progress: 13%
Progress: 14%
Progress: 15%
Progress: 16%
Progress: 17%
Progress: 18%
Progress: 19%
Progress: 20%
Progress: 21%
Progress: 22%
Progress: 23%
Progress: 24%
Progress: 25%
Progress: 26%
Progress: 27%
Progress: 28%
Progress: 29%
Progress: 30%
Progress: 31%
Progress: 32%
Progress: 33%
Progress: 34%
Progress: 35%
Progress: 36%
Progress: 37%
Progress: 38%
Progress: 39%
Progress: 40%
Progress: 41%
Progress: 42%
Progress: 43%
Progress: 44%
Progress: 45%
Progress: 46%
Progress: 47%
Progress: 48%
Progress: 49%
Progress: 50%
Progr

In [7]:
# # plot an image with rasterio
# flow_tif = os.path.join(temp_dir, "flow_dem.tif")
# img = rasterio.open(flow_tif)
# show(img)

#### Generate Flow Accumulation

In [8]:
flow_accum = os.path.join(temp_dir, 'flow_accum.tif')
flow_dem = os.path.join(temp_dir, 'flow_den.tif')

if not os.path.exists(flow_accum):
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        future = executor.submit(flow_accumulation, flow_direction, temp_dir)

Working directory: data/temp_dir
********************************************
* Welcome to d8_flow_accum                 *
* Powered by Whitebox Workflows for Python *
* www.whiteboxgeo.com                      *
********************************************
Flow directions: 0%
Flow directions: 1%
Flow directions: 2%
Flow directions: 3%
Flow directions: 4%
Flow directions: 5%
Flow directions: 6%
Flow directions: 7%
Flow directions: 8%
Flow directions: 9%
Flow directions: 10%
Flow directions: 11%
Flow directions: 12%
Flow directions: 13%
Flow directions: 14%
Flow directions: 15%
Flow directions: 16%
Flow directions: 17%
Flow directions: 18%
Flow directions: 19%
Flow directions: 20%
Flow directions: 21%
Flow directions: 22%
Flow directions: 23%
Flow directions: 24%
Flow directions: 25%
Flow directions: 26%
Flow directions: 27%
Flow directions: 28%
Flow directions: 29%
Flow directions: 30%
Flow directions: 31%
Flow directions: 32%
Flow directions: 33%
Flow directions: 34%
Flow directions: 